In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
train_data = pd.read_csv("/kaggle/input/analytics-vidhya-ltfs-data-science-finhack-3/Data/Data/train_Data.csv")
train_bureau = pd.read_csv("/kaggle/input/analytics-vidhya-ltfs-data-science-finhack-3/Data/Data/train_bureau.csv")

test_data = pd.read_csv("/kaggle/input/analytics-vidhya-ltfs-data-science-finhack-3/Data/Data/test_Data.csv")
test_bureau = pd.read_csv("/kaggle/input/analytics-vidhya-ltfs-data-science-finhack-3/Data/Data/test_bureau.csv")

# Working on Bureau Data

In [ ]:
train_bureau = train_bureau.drop_duplicates()
test_bureau = test_bureau.drop_duplicates()

In [ ]:
train_data['source'] = 'train'
test_data['source'] = 'test'

train_bureau['source'] = 'train'
test_bureau['source'] = 'test'

In [ ]:
bureau = pd.concat([train_bureau, test_bureau])
data = pd.concat([train_data, test_data])

In [ ]:
for col in bureau.columns:
    if bureau[col].dtype=='object':
        print(bureau[col].value_counts())
        print('')

Can drop MATCH-TYPE, since very few are of secondary type
fix account status
fix contributor type

In [ ]:
bureau['ACCT-TYPE'] = bureau['ACCT-TYPE'].replace(bureau['ACCT-TYPE'].value_counts()[bureau['ACCT-TYPE'].value_counts()/bureau.shape[0]*100<3].index, 'Rest')
bureau['ACCT-TYPE'].value_counts()

In [ ]:
bureau['CONTRIBUTOR-TYPE'] = bureau['CONTRIBUTOR-TYPE'].replace(['HFC', 'MFI', 'FRB',
       'SFB', 'ARC', 'OFI' ,'CCC'], 'OTHERS')
bureau['CONTRIBUTOR-TYPE'].value_counts()

In [ ]:
bureau['ACCOUNT-STATUS'] = bureau['ACCOUNT-STATUS'].replace(['SUIT FILED (WILFUL DEFAULT)',
       'Written Off', 'Suit Filed', 'Restructured', 'Settled',
       'WILFUL DEFAULT', 'Cancelled', 'Sold/Purchased'], 'Written Off')
bureau['ACCOUNT-STATUS'].value_counts()

In [ ]:
bureau.isnull().sum()[bureau.isnull().sum()>0]/bureau.shape[0]*100

In [ ]:
#drop columns with greater than 20% null values
bureau.drop(bureau.isnull().sum()[bureau.isnull().sum()/bureau.shape[0]*100>20].index, axis=1, inplace=True)

In [ ]:
bureau.isnull().sum()[bureau.isnull().sum()>0]/bureau.shape[0]*100

In [ ]:
#can drop hist columns for now
bureau = bureau.drop(['REPORTED DATE - HIST', 'DPD - HIST', 'CUR BAL - HIST',
       'AMT OVERDUE - HIST', 'AMT PAID - HIST'], axis=1)

In [ ]:
bureau['DISBURSED-DT'] = pd.to_datetime(bureau['DISBURSED-DT'])
bureau['DATE-REPORTED'] = pd.to_datetime(bureau['DATE-REPORTED'])
bureau['DATE-REPORTED'].max(), bureau['DISBURSED-DT'].max()

In [ ]:
bureau['DATE-REPORTED'] = bureau['DATE-REPORTED'].fillna(bureau['DATE-REPORTED'].max())
bureau['DISBURSED-DT'] = bureau['DISBURSED-DT'].fillna(bureau['DATE-REPORTED'])

In [ ]:
bureau.drop('MATCH-TYPE', axis=1, inplace=True)

In [ ]:
bureau['CURRENT-BAL'] = bureau['CURRENT-BAL'].apply(lambda x: str(x).replace(',', ''))
bureau['DISBURSED-AMT/HIGH CREDIT'] = bureau['DISBURSED-AMT/HIGH CREDIT'].apply(lambda x: str(x).replace(',', ''))
bureau['WRITE-OFF-AMT'] = bureau['WRITE-OFF-AMT'].apply(lambda x: str(x).replace(',', ''))

In [ ]:
#bureau['CURRENT-BAL'] = bureau['CURRENT-BAL'].fillna('0')
bureau.isnull().sum()

In [ ]:
for col in bureau.columns[-4:-1]:
    print(col)
    bureau[col] = bureau[col].replace('nan', 0)
    bureau[col] = bureau[col].apply(lambda x: float(x))

In [ ]:
bureau.head()

In [ ]:
bureau = pd.get_dummies(data=bureau, columns=['ACCT-TYPE', 'OWNERSHIP-IND', 'CONTRIBUTOR-TYPE', 
                                    'ACCOUNT-STATUS'])

In [ ]:
bureau.columns

In [ ]:
bureau_data = bureau.groupby(['ID', 'source']).sum().reset_index()
bureau_data.to_csv('bureau_data.csv', index=False)

In [ ]:
bureau_data.head()

In [ ]:
df = pd.read_csv('bureau_data.csv')
df.head()

In [ ]:
df.tail()

# Working on demographic data

In [ ]:
#target variable is "Top-up Month"

In [ ]:
# can drop unwanted columns
data = data.drop(['Area', 'BranchID', 'City', 'ZiPCODE'], axis=1)

In [ ]:
data['DisbursalDate'] = pd.to_datetime(data['DisbursalDate'])
data['MaturityDAte'] = pd.to_datetime(data['MaturityDAte'])
data['AuthDate'] = pd.to_datetime(data['AuthDate'])

In [ ]:
for col in data.columns:
    if data[col].dtype=='object':
        print(data[col].value_counts())
        print('')

In [ ]:
data['State'] = data['State'].replace(data['State'].value_counts()[data['State'].value_counts()/data.shape[0]*100<3].index, 'Rest')
data['State'].value_counts()

In [ ]:
data['PaymentMode'] = data['PaymentMode'].replace(data['PaymentMode'].value_counts()[data['PaymentMode'].value_counts()/data.shape[0]*100<3].index, 'Rest')
data['PaymentMode'].value_counts()

In [ ]:
data.isnull().sum()

In [ ]:
cat_cols = ['ManufacturerID', 'SupplierID', 'AssetID']
for col in cat_cols:
    print('value counts')
    print(data[col].value_counts())
    print('number of unique values', data[col].nunique())
    print()

In [ ]:
data.drop(['SupplierID', 'AssetID'], axis=1, inplace=True)

In [ ]:
data['ManufacturerID'] = data['ManufacturerID'].replace(data['ManufacturerID'].value_counts()[data['ManufacturerID'].value_counts()/data.shape[0]*100<3].index, 'Rest')
data['ManufacturerID'].value_counts()

In [ ]:
data.head()

In [ ]:
data['MonthlyIncome'] = data['MonthlyIncome'].fillna(0)
data['SEX'] = data['SEX'].fillna(data['SEX'].mode()[0])
data['AGE'] = data['AGE'].fillna(data['AGE'].mode()[0])
data['ManufacturerID'] = data['ManufacturerID'].fillna(data['ManufacturerID'].mode()[0])
data['MaturityDAte'] = data['MaturityDAte'].fillna(data['MaturityDAte'].mode()[0])

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
data.columns

In [ ]:
le = LabelEncoder()
cat_cols = ['Frequency', 'InstlmentMode', 'LoanStatus', 'PaymentMode', 'SEX', 'State']

for col in cat_cols:
    print(col)
    data[col] = le.fit_transform(data[col])

In [ ]:
num_cols = ['AssetCost', 'AmountFinance', 'DisbursalAmount', 'EMI', 'MonthlyIncome']
#scaler = StandardScaler()
"""for col in num_cols:
    print(col)
    data[col] = scaler.fit_transform(np.array(data[col]).reshape(-1,1))"""

for col in num_cols:
    data[col] = np.log(1+data[col])
    print(col)

In [ ]:
data.drop(['DisbursalDate', 'MaturityDAte', 'AuthDate'], axis=1, inplace=True)
data.head()

In [ ]:
data['EMItoIncome'] = data['EMI']/data['MonthlyIncome']
data['AssettoIncome'] = data['AssetCost']/(data['MonthlyIncome']*12)
data['DifferenceAmount'] = data['AmountFinance'] - data['DisbursalAmount']
data['YearsOfService'] = 60 - data['AGE']

In [ ]:
num_cols = ['AssetCost', 'AmountFinance', 'DisbursalAmount', 'EMI', 'MonthlyIncome']
#scaler = StandardScaler()
"""for col in num_cols:
    print(col)
    data[col] = scaler.fit_transform(np.array(data[col]).reshape(-1,1))"""

for col in num_cols:
    data[col] = np.log(1+data[col])
    print(col)

In [ ]:
data.head()

In [ ]:
data.drop('AGE', 1, inplace=True)

In [ ]:
data.to_csv('cleaned_data.csv', index=False)

In [ ]:
final = pd.merge(data, df, on=['ID', 'source'], how='inner')
final.head()

In [ ]:
train_df = final[final['source']=='train']
test_df = final[final['source']=='test']

In [ ]:
train_df.drop('source',1, inplace=True)
test_df.drop(['source', 'Top-up Month'],1, inplace=True)

In [ ]:
test_df.shape, test_data.shape

In [ ]:
from sklearn.model_selection import train_test_split

train = train_df.drop('ID', 1)
X = train_df.drop(['ID', 'Top-up Month'], 1)
y = train_df['Top-up Month']

train_X, test_X, train_y ,test_y = train_test_split(X, y, test_size=0.2, random_state=42)

test = test_df.drop(['ID'], 1)
train_X.shape, train_y.shape, test.shape

In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

In [ ]:
!pip install pycaret

In [ ]:
from pycaret.classification import *

In [ ]:
clf = setup(data = train, target = 'Top-up Month', session_id=123)

In [ ]:
#best_model = compare_models()

In [ ]:
models()

In [ ]:
cat = create_model('catboost')
plot_model(cat, plot='feature')
test_pred = predict_model(cat, data=test)
subm = pd.DataFrame({'ID': test_df['ID'], 'Top-up Month': test_pred['Label']})
subm.to_csv('pycaret_catboost.csv', index=False)